In [1]:
from sklearn.ensemble import RandomForestClassifier
#import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
#import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC
from tensorflow.keras.callbacks import EarlyStopping
import joblib

In [2]:
def ROC_AUC(y_true, y_score):
	auc = roc_auc_score(y_true, y_score)
	return auc

In [3]:
fingerprint_list = ['MorganFP', 'RDkitFP', 'AtomPairFP', 'TorsionFP', 'AvalonFP', 
'EstateFP', 'MACCSFP', 'PharmacoErGFP', 'PharmacoPFP', 'PubChemFP', 'MHFP6', 'MAP4']
descriptor_list = ['Property', 'Constitution', 'Autocorr', 'Fragment', 'Charge', 
'Estate', 'MOE', 'Connectivity', 'Topology', 'Kappa', 'Path', 'Matrix', 'InfoContent']

In [4]:
def get_ext_rm_sim_Xye_fp_desc(ext_file,feature):
    #feature='AtomPairFP'
    ext_base = ext_file.replace('.csv','')
    df = pd.read_csv(ext_file)
    smiles_col = df.columns[0]
    values_col = df.columns[1]
    ext_feat_file = f"fp2/{ext_base}_{feature}.csv"
    dff = pd.read_csv(ext_feat_file)
    print('values_col = ',values_col)
    #feats_col = df.columns[2:]
    MASK = -1
    #y = df[values_col].astype('int').fillna(MASK).values
    y_e = df[values_col].astype('int').fillna(MASK).values
    #X = df[feats_col].values
    X_e = dff.values
    n_feats = len(dff.columns)
    return X_e,y_e

In [5]:
def get_ext_Xye_fp_desc(ext_file,feature):
    #feature='AtomPairFP'
    ext_base = ext_file.replace('.csv','')
    df = pd.read_csv(ext_file)
    smiles_col = df.columns[0]
    values_col = df.columns[1]
    if feature in fingerprint_list:
        ext_feat_file = f"fp/{ext_base}_{feature}.csv"
    elif feature in descriptor_list:
        ext_feat_file = f"fp/{ext_base}_{feature}S.csv"
    dff = pd.read_csv(ext_feat_file)
    print('values_col = ',values_col)
    #feats_col = df.columns[2:]
    MASK = -1
    #y = df[values_col].astype('int').fillna(MASK).values
    y_e = df[values_col].astype('int').fillna(MASK).values
    #X = df[feats_col].values
    X_e = dff.values
    n_feats = len(dff.columns)
    return X_e,y_e

# load pretrained xgboost model and make prediction

In [7]:
def load_xgb_and_predict(feature,num_folds):   
    for i in range(num_folds): 
        #print(f'processing fold: {i}')
        model_file=f"xgb/model_{feature}_fold{i}.job"
        xgboost_model = joblib.load(model_file)
        y_prob_e =  xgboost_model.predict_proba(X_e)
        e_roc_auc = ROC_AUC(y_e,y_prob_e[:, 1])
        print('e_roc_auc = ', e_roc_auc)
        if not os.path.exists('xgb'):os.mkdir('xgb')
        pd.DataFrame(y_prob_e[:,1],columns=['prob']).to_csv(f'xgb/p_{ext_base}_{feature}_fold{i}.csv',index=False) 
    return

In [15]:
num_folds = 5;
ext_files = ['Ext_rm_sim_0_6.csv','Ext_rm_sim_0_7.csv','Ext_rm_sim_0_8.csv','Ext_rm_sim_0_9.csv']
for ext_file in ext_files:
    ext_base = ext_file.replace('.csv','')
    for feature in fingerprint_list+descriptor_list:
        X_e,y_e = get_ext_rm_sim_Xye_fp_desc(ext_file,feature)
        print(f'processing ext_file: {ext_file}; feat: {feature}')
        load_xgb_and_predict(feature,num_folds)

values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: MorganFP
e_roc_auc =  0.1111111111111111
e_roc_auc =  0.25925925925925924
e_roc_auc =  0.18518518518518517
e_roc_auc =  0.0740740740740741
e_roc_auc =  0.22222222222222224
values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: RDkitFP
e_roc_auc =  0.11111111111111116
e_roc_auc =  0.11111111111111116
e_roc_auc =  0.11111111111111116
e_roc_auc =  0.11111111111111116
e_roc_auc =  0.11111111111111116
values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: AtomPairFP
e_roc_auc =  0.22222222222222224
e_roc_auc =  0.18518518518518517
e_roc_auc =  0.14814814814814817
e_roc_auc =  0.14814814814814817
e_roc_auc =  0.18518518518518517
values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: TorsionFP
e_roc_auc =  0.0
e_roc_auc =  0.07407407407407407
e_roc_auc =  0.0
e_roc_auc =  0.0
e_roc_auc =  0.14814814814814814
values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: AvalonFP
e_roc_au

e_roc_auc =  0.6105263157894737
e_roc_auc =  0.7042105263157895
e_roc_auc =  0.6357894736842105
values_col =  label
processing ext_file: Ext_rm_sim_0_7.csv; feat: MAP4
e_roc_auc =  0.7063157894736842
e_roc_auc =  0.74
e_roc_auc =  0.6494736842105263
e_roc_auc =  0.6378947368421053
e_roc_auc =  0.6231578947368421
values_col =  label
processing ext_file: Ext_rm_sim_0_7.csv; feat: Property
e_roc_auc =  0.488421052631579
e_roc_auc =  0.4842105263157896
e_roc_auc =  0.5294736842105263
e_roc_auc =  0.3915789473684211
e_roc_auc =  0.40210526315789474
values_col =  label
processing ext_file: Ext_rm_sim_0_7.csv; feat: Constitution
e_roc_auc =  0.4952631578947369
e_roc_auc =  0.506842105263158
e_roc_auc =  0.633157894736842
e_roc_auc =  0.5257894736842105
e_roc_auc =  0.3942105263157895
values_col =  label
processing ext_file: Ext_rm_sim_0_7.csv; feat: Autocorr
e_roc_auc =  0.6263157894736842
e_roc_auc =  0.5663157894736842
e_roc_auc =  0.6094736842105263
e_roc_auc =  0.43052631578947376
e_roc_a

e_roc_auc =  0.6131924882629108
e_roc_auc =  0.641643192488263
e_roc_auc =  0.6532863849765258
values_col =  label
processing ext_file: Ext_rm_sim_0_8.csv; feat: Kappa
e_roc_auc =  0.5379812206572769
e_roc_auc =  0.5292488262910797
e_roc_auc =  0.4884037558685447
e_roc_auc =  0.4623943661971831
e_roc_auc =  0.5554460093896714
values_col =  label
processing ext_file: Ext_rm_sim_0_8.csv; feat: Path
e_roc_auc =  0.5757276995305164
e_roc_auc =  0.5596713615023474
e_roc_auc =  0.5678403755868545
e_roc_auc =  0.5280281690140844
e_roc_auc =  0.5122535211267605
values_col =  label
processing ext_file: Ext_rm_sim_0_8.csv; feat: Matrix
e_roc_auc =  0.7879812206572769
e_roc_auc =  0.7067605633802817
e_roc_auc =  0.6658215962441315
e_roc_auc =  0.6251643192488263
e_roc_auc =  0.7822535211267605
values_col =  label
processing ext_file: Ext_rm_sim_0_8.csv; feat: InfoContent
e_roc_auc =  0.6590140845070422
e_roc_auc =  0.6188262910798122
e_roc_auc =  0.6052112676056338
e_roc_auc =  0.5818309859154929

In [16]:
num_folds = 5;
ext_file = 'Ext.csv'
ext_base = ext_file.replace('.csv','')
for feature in fingerprint_list+descriptor_list:
    X_e,y_e = get_ext_Xye_fp_desc(ext_file,feature)
    print(f'processing ext_file: {ext_file}; feat: {feature}')
    load_xgb_and_predict(feature,num_folds)

values_col =  label
processing ext_file: Ext.csv; feat: MorganFP
e_roc_auc =  0.8363290026374914
e_roc_auc =  0.8626062322946175
e_roc_auc =  0.8580476050926378
e_roc_auc =  0.8632900263749146
e_roc_auc =  0.8700302823092704
values_col =  label
processing ext_file: Ext.csv; feat: RDkitFP
e_roc_auc =  0.8046791052066035
e_roc_auc =  0.8310377389209078
e_roc_auc =  0.8193806779329882
e_roc_auc =  0.7831395916772491
e_roc_auc =  0.7927778320471491
values_col =  label
processing ext_file: Ext.csv; feat: AtomPairFP
e_roc_auc =  0.9047735339129303
e_roc_auc =  0.917244635472632
e_roc_auc =  0.8967959363094657
e_roc_auc =  0.9051317117645143
e_roc_auc =  0.8878089283969913
values_col =  label
processing ext_file: Ext.csv; feat: TorsionFP
e_roc_auc =  0.8144475920679888
e_roc_auc =  0.8035394484061087
e_roc_auc =  0.8317052521897692
e_roc_auc =  0.8002181628732377
e_roc_auc =  0.8391944254501645
values_col =  label
processing ext_file: Ext.csv; feat: AvalonFP
e_roc_auc =  0.7685519846309141
e_

# load pretrained fcnn model and predict

In [17]:
# seem the keras model can only run once inside a function, otherwise will get GPU errors.
def load_one_keras_fcnn_model_pred(feature,i):
    ## save model
    model_save_name = f'fcnn/model_{feature}_fold{i}.h5'
    #model.save(model_save_name)
    model = tf.keras.models.load_model(model_save_name)
    ## prediction
    y_prob_e=model.predict(X_e)  ## no need other parameters here
    e_roc_auc = ROC_AUC(y_e,y_prob_e[:, 0])
    print('e_roc_auc = ', e_roc_auc)    
    ## save predition
    if not os.path.exists('fcnn'):os.mkdir('fcnn')
    pd.DataFrame(y_prob_e[:,0],columns=['prob']).to_csv(f'fcnn/p_{ext_base}_{feature}_fold{i}.csv',index=False) 
    return

In [12]:
os.environ["CUDA_VISIBLE_DEVICES"]="2"   
num_folds = 5;
ext_files = ['Ext_rm_sim_0_6.csv','Ext_rm_sim_0_7.csv','Ext_rm_sim_0_8.csv','Ext_rm_sim_0_9.csv']
for ext_file in ext_files:
    ext_base = ext_file.replace('.csv','')
    for feature in fingerprint_list+descriptor_list:
        X_e,y_e = get_ext_rm_sim_Xye_fp_desc(ext_file,feature)
        print(f'processing ext_file: {ext_file}; feat: {feature}')
        for i in range(num_folds): 
            load_one_keras_fcnn_model_pred(feature,i)

values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: MorganFP


2023-01-24 00:20:21.259079: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 00:20:23.998093: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7971 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:25:00.0, compute capability: 8.6
2023-01-24 00:20:23.999552: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 8037 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:61:00.0, compute capability: 8.6
2023-01-24 00:20:24.000878: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/de

1/1 [==============================] - 1s 793ms/step
e_roc_auc =  0.4074074074074074
1/1 [==============================] - 0s 67ms/step


2023-01-24 00:20:25.055315: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


e_roc_auc =  0.48148148148148145
1/1 [==============================] - 0s 65ms/step
e_roc_auc =  0.48148148148148145
1/1 [==============================] - 0s 66ms/step
e_roc_auc =  0.48148148148148145
1/1 [==============================] - 0s 67ms/step
e_roc_auc =  0.5925925925925926
values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: RDkitFP
1/1 [==============================] - 0s 73ms/step
e_roc_auc =  0.11111111111111116
1/1 [==============================] - 0s 66ms/step
e_roc_auc =  0.11111111111111116
1/1 [==============================] - 0s 67ms/step
e_roc_auc =  0.11111111111111116
1/1 [==============================] - 0s 68ms/step
e_roc_auc =  0.11111111111111116
1/1 [==============================] - 0s 67ms/step
e_roc_auc =  0.0740740740740741
values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: AtomPairFP
1/1 [==============================] - 0s 67ms/step
e_roc_auc =  0.5925925925925926
1/1 [==============================] - 0s 67ms/step


1/1 [==============================] - 0s 66ms/step
e_roc_auc =  0.5925925925925926
1/1 [==============================] - 0s 68ms/step
e_roc_auc =  0.4444444444444445
1/1 [==============================] - 0s 66ms/step
e_roc_auc =  0.4814814814814815
1/1 [==============================] - 0s 73ms/step
e_roc_auc =  0.33333333333333337
1/1 [==============================] - 0s 77ms/step
e_roc_auc =  0.29629629629629634
values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: Fragment
1/1 [==============================] - 0s 67ms/step
e_roc_auc =  0.11111111111111116
1/1 [==============================] - 0s 67ms/step
e_roc_auc =  0.22222222222222224
1/1 [==============================] - 0s 67ms/step
e_roc_auc =  0.11111111111111116
1/1 [==============================] - 0s 66ms/step
e_roc_auc =  0.11111111111111116
1/1 [==============================] - 0s 66ms/step
e_roc_auc =  0.11111111111111116
values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: Charge
1/1

2/2 [==============================] - 0s 2ms/step
e_roc_auc =  0.17842105263157895
2/2 [==============================] - 0s 2ms/step
e_roc_auc =  0.15526315789473685
values_col =  label
processing ext_file: Ext_rm_sim_0_7.csv; feat: MACCSFP
2/2 [==============================] - 0s 2ms/step
e_roc_auc =  0.3026315789473685
2/2 [==============================] - 0s 2ms/step
e_roc_auc =  0.3173684210526316
2/2 [==============================] - 0s 2ms/step
e_roc_auc =  0.3468421052631579
2/2 [==============================] - 0s 2ms/step
e_roc_auc =  0.2573684210526316
2/2 [==============================] - 0s 2ms/step
e_roc_auc =  0.3068421052631579
values_col =  label
processing ext_file: Ext_rm_sim_0_7.csv; feat: PharmacoErGFP
2/2 [==============================] - 0s 2ms/step
e_roc_auc =  0.7547368421052632
2/2 [==============================] - 0s 2ms/step
e_roc_auc =  0.7494736842105264
2/2 [==============================] - 0s 2ms/step
e_roc_auc =  0.7136842105263158
2/2 [=======

values_col =  label
processing ext_file: Ext_rm_sim_0_8.csv; feat: Autocorr
7/7 [==============================] - 0s 2ms/step
e_roc_auc =  0.7091079812206572
7/7 [==============================] - 0s 1ms/step
e_roc_auc =  0.6724882629107982
7/7 [==============================] - 0s 1ms/step
e_roc_auc =  0.6493896713615023
7/7 [==============================] - 0s 1ms/step
e_roc_auc =  0.6454460093896713
7/7 [==============================] - 0s 1ms/step
e_roc_auc =  0.45924882629107977
values_col =  label
processing ext_file: Ext_rm_sim_0_8.csv; feat: Fragment
7/7 [==============================] - 0s 1ms/step
e_roc_auc =  0.4056338028169014
7/7 [==============================] - 0s 1ms/step
e_roc_auc =  0.41737089201877936
7/7 [==============================] - 0s 1ms/step
e_roc_auc =  0.3676056338028169
7/7 [==============================] - 0s 1ms/step
e_roc_auc =  0.37953051643192487
7/7 [==============================] - 0s 1ms/step
e_roc_auc =  0.39333333333333337
values_col =  

e_roc_auc =  0.573539588845919
12/12 [==============================] - 0s 1ms/step
e_roc_auc =  0.5649501323020558
12/12 [==============================] - 0s 1ms/step
e_roc_auc =  0.5463464278444942
values_col =  label
processing ext_file: Ext_rm_sim_0_9.csv; feat: MACCSFP
12/12 [==============================] - 0s 1ms/step
e_roc_auc =  0.7049460614695704
12/12 [==============================] - 0s 1ms/step
e_roc_auc =  0.6980256462446571
12/12 [==============================] - 0s 1ms/step
e_roc_auc =  0.7032363118257683
12/12 [==============================] - 0s 1ms/step
e_roc_auc =  0.6821901078770608
12/12 [==============================] - 0s 1ms/step
e_roc_auc =  0.6993690209647873
values_col =  label
processing ext_file: Ext_rm_sim_0_9.csv; feat: PharmacoErGFP
12/12 [==============================] - 0s 1ms/step
e_roc_auc =  0.8158966008548748
12/12 [==============================] - 0s 1ms/step
e_roc_auc =  0.8059230612660289
12/12 [==============================] - 0s 1ms/

12/12 [==============================] - 0s 1ms/step
e_roc_auc =  0.7716873600651333
12/12 [==============================] - 0s 1ms/step
e_roc_auc =  0.7649297781396295
12/12 [==============================] - 0s 1ms/step
e_roc_auc =  0.6667005902707103
12/12 [==============================] - 0s 1ms/step
e_roc_auc =  0.7712395684917567
12/12 [==============================] - 0s 1ms/step
e_roc_auc =  0.6511093018522288
values_col =  label
processing ext_file: Ext_rm_sim_0_9.csv; feat: Matrix
12/12 [==============================] - 0s 1ms/step
e_roc_auc =  0.7074292692855688
12/12 [==============================] - 0s 1ms/step
e_roc_auc =  0.7067372277630776
12/12 [==============================] - 0s 1ms/step
e_roc_auc =  0.5366578465296153
12/12 [==============================] - 0s 1ms/step
e_roc_auc =  0.6757581925503765
12/12 [==============================] - 0s 1ms/step
e_roc_auc =  0.6680643191532669
values_col =  label
processing ext_file: Ext_rm_sim_0_9.csv; feat: InfoConte

In [18]:
os.environ["CUDA_VISIBLE_DEVICES"]="2"   
num_folds = 5;
ext_file = 'Ext.csv'
ext_base = ext_file.replace('.csv','')
for feature in fingerprint_list+descriptor_list:
    X_e,y_e = get_ext_Xye_fp_desc(ext_file,feature)
    print(f'processing ext_file: {ext_file}; feat: {feature}')
    for i in range(num_folds): 
        load_one_keras_fcnn_model_pred(feature,i)

values_col =  label
processing ext_file: Ext.csv; feat: MorganFP
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.8613688906255088
14/14 [==============================] - 0s 2ms/step
e_roc_auc =  0.8419621633942236
14/14 [==============================] - 0s 2ms/step
e_roc_auc =  0.8425157109830355
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.8240858324378888
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.8300120478004624
values_col =  label
processing ext_file: Ext.csv; feat: RDkitFP
14/14 [==============================] - 0s 2ms/step
e_roc_auc =  0.7915079287551692
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.8295399042688287
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.785418905278239
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.7555110546709648
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.7524014196867572
values_col =  label
pr

free GPU memory

In [19]:
# free GPU memory
from numba import cuda 
device = cuda.get_current_device()
device.reset()
cuda.close()

LogisticRegression

#  load pretrained log model and predict

In [20]:
def load_one_log_model_and_predict(feature,num_folds):    
    #print(f'processing fold: {i}')
    model_file=f"log/model_{feature}_fold{i}.job"
    Log_model = joblib.load(model_file)
    y_prob_e=Log_model.predict_proba(X_e)[:,1]  ## no need other parameters here
    e_roc_auc = ROC_AUC(y_e,y_prob_e)    
    print('e_roc_auc = ', e_roc_auc)
    if not os.path.exists('log'):os.mkdir('log')
    pd.DataFrame(y_prob_e,columns=['prob']).to_csv(f'log/p_{ext_base}_{feature}_fold{i}.csv',index=False)
    return

In [21]:
num_folds = 5;
ext_files = ['Ext_rm_sim_0_6.csv','Ext_rm_sim_0_7.csv','Ext_rm_sim_0_8.csv','Ext_rm_sim_0_9.csv']
for ext_file in ext_files:
    ext_base = ext_file.replace('.csv','')
    for feature in fingerprint_list+descriptor_list:
        X_e,y_e = get_ext_rm_sim_Xye_fp_desc(ext_file,feature)
        print(f'processing ext_file: {ext_file}; feat: {feature}')
        for i in range(num_folds): 
            load_one_log_model_and_predict(feature,num_folds)

values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: MorganFP
e_roc_auc =  0.18518518518518523
e_roc_auc =  0.0740740740740741
e_roc_auc =  0.0740740740740741
e_roc_auc =  0.0740740740740741
e_roc_auc =  0.22222222222222224
values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: RDkitFP
e_roc_auc =  0.1111111111111111
e_roc_auc =  0.11111111111111116
e_roc_auc =  0.03703703703703705
e_roc_auc =  0.0740740740740741
e_roc_auc =  0.0740740740740741
values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: AtomPairFP
e_roc_auc =  0.4444444444444445
e_roc_auc =  0.8518518518518519
e_roc_auc =  0.4444444444444444
e_roc_auc =  0.7407407407407407
e_roc_auc =  0.6296296296296295
values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: TorsionFP
e_roc_auc =  0.0
e_roc_auc =  0.0
e_roc_auc =  0.0
e_roc_auc =  0.0
e_roc_auc =  0.0
values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: AvalonFP
e_roc_auc =  0.4444444444444444
e_roc_auc =  0.51

values_col =  label
processing ext_file: Ext_rm_sim_0_7.csv; feat: Fragment
e_roc_auc =  0.1542105263157895
e_roc_auc =  0.20157894736842105
e_roc_auc =  0.1615789473684211
e_roc_auc =  0.15842105263157896
e_roc_auc =  0.18789473684210528
values_col =  label
processing ext_file: Ext_rm_sim_0_7.csv; feat: Charge
e_roc_auc =  0.5757894736842105
e_roc_auc =  0.616842105263158
e_roc_auc =  0.5989473684210527
e_roc_auc =  0.5842105263157895
e_roc_auc =  0.6231578947368421
values_col =  label
processing ext_file: Ext_rm_sim_0_7.csv; feat: Estate
e_roc_auc =  0.5421052631578948
e_roc_auc =  0.5178947368421053
e_roc_auc =  0.5357894736842106
e_roc_auc =  0.48315789473684206
e_roc_auc =  0.4757894736842105
values_col =  label
processing ext_file: Ext_rm_sim_0_7.csv; feat: MOE
e_roc_auc =  0.5263157894736842
e_roc_auc =  0.5663157894736841
e_roc_auc =  0.5547368421052631
e_roc_auc =  0.4589473684210526
e_roc_auc =  0.5284210526315789
values_col =  label
processing ext_file: Ext_rm_sim_0_7.csv; f

e_roc_auc =  0.8044982698961938
values_col =  label
processing ext_file: Ext_rm_sim_0_9.csv; feat: RDkitFP
e_roc_auc =  0.6347242010991248
e_roc_auc =  0.6526155098717688
e_roc_auc =  0.5482800732749847
e_roc_auc =  0.5855688988398128
e_roc_auc =  0.6037655200488501
values_col =  label
processing ext_file: Ext_rm_sim_0_9.csv; feat: AtomPairFP
e_roc_auc =  0.8849175656421737
e_roc_auc =  0.917524933848972
e_roc_auc =  0.8955831467535111
e_roc_auc =  0.9134337472013027
e_roc_auc =  0.8683085691023813
values_col =  label
processing ext_file: Ext_rm_sim_0_9.csv; feat: TorsionFP
e_roc_auc =  0.6139222470995319
e_roc_auc =  0.6239975575005088
e_roc_auc =  0.58719723183391
e_roc_auc =  0.5238754325259516
e_roc_auc =  0.5886016690413189
values_col =  label
processing ext_file: Ext_rm_sim_0_9.csv; feat: AvalonFP
e_roc_auc =  0.62937105638103
e_roc_auc =  0.6258294321188683
e_roc_auc =  0.593038876450234
e_roc_auc =  0.521188683085691
e_roc_auc =  0.5728068389985752
values_col =  label
processin

# #  load pretrained svm model and predict

In [10]:
def load_svm_model_and_predict(feature,num_folds):  
    for i in range(num_folds): 
        model_file=f"svm/model_{feature}_fold{i}.job"
        svm_clf = joblib.load(model_file)
        y_prob_e=svm_clf.predict_proba(X_e)[:,1]  ## no need other parameters here
        e_roc_auc = ROC_AUC(y_e,y_prob_e)
        print('e_roc_auc = ', e_roc_auc)
        if not os.path.exists('svm'):os.mkdir('svm')
        pd.DataFrame(y_prob_e,columns=['prob']).to_csv(f'svm/p_{ext_base}_{feature}_fold{i}.csv',index=False)
    return

In [12]:
from sklearn import svm
num_folds = 5;
ext_files = ['Ext_rm_sim_0_6.csv','Ext_rm_sim_0_7.csv','Ext_rm_sim_0_8.csv','Ext_rm_sim_0_9.csv']
for ext_file in ext_files:
    ext_base = ext_file.replace('.csv','')
    for feature in fingerprint_list+descriptor_list:
        X_e,y_e = get_ext_rm_sim_Xye_fp_desc(ext_file,feature)
        print(f'processing ext_file: {ext_file}; feat: {feature}')
        load_svm_model_and_predict(feature,num_folds)

values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: MorganFP
e_roc_auc =  0.03703703703703705
e_roc_auc =  0.0
e_roc_auc =  0.0
e_roc_auc =  0.0
e_roc_auc =  0.07407407407407407
values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: RDkitFP
e_roc_auc =  0.1111111111111111
e_roc_auc =  0.11111111111111116
e_roc_auc =  0.0740740740740741
e_roc_auc =  0.11111111111111116
e_roc_auc =  0.0740740740740741
values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: AtomPairFP
e_roc_auc =  0.5555555555555556
e_roc_auc =  0.8148148148148149
e_roc_auc =  0.5555555555555556
e_roc_auc =  0.5555555555555556
e_roc_auc =  0.7777777777777778
values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: TorsionFP
e_roc_auc =  0.0
e_roc_auc =  0.0
e_roc_auc =  0.03703703703703705
e_roc_auc =  0.0
e_roc_auc =  0.0
values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: AvalonFP
e_roc_auc =  0.7777777777777779
e_roc_auc =  0.7407407407407407
e_roc_auc =  0

e_roc_auc =  0.6263157894736843
e_roc_auc =  0.5084210526315789
values_col =  label
processing ext_file: Ext_rm_sim_0_7.csv; feat: MAP4
e_roc_auc =  0.7757894736842106
e_roc_auc =  0.7178947368421054
e_roc_auc =  0.751578947368421
e_roc_auc =  0.771578947368421
e_roc_auc =  0.7557894736842106
values_col =  label
processing ext_file: Ext_rm_sim_0_7.csv; feat: Property
e_roc_auc =  0.47052631578947374
e_roc_auc =  0.3873684210526316
e_roc_auc =  0.4105263157894737
e_roc_auc =  0.3915789473684211
e_roc_auc =  0.37473684210526315
values_col =  label
processing ext_file: Ext_rm_sim_0_7.csv; feat: Constitution
e_roc_auc =  0.34684210526315795
e_roc_auc =  0.36052631578947375
e_roc_auc =  0.3394736842105263
e_roc_auc =  0.3015789473684211
e_roc_auc =  0.19631578947368422
values_col =  label
processing ext_file: Ext_rm_sim_0_7.csv; feat: Autocorr
e_roc_auc =  0.6094736842105263
e_roc_auc =  0.5768421052631578
e_roc_auc =  0.5947368421052631
e_roc_auc =  0.5757894736842105
e_roc_auc =  0.572631

e_roc_auc =  0.591549295774648
e_roc_auc =  0.6242253521126762
e_roc_auc =  0.5878873239436619
e_roc_auc =  0.5491079812206573
e_roc_auc =  0.5536150234741785
values_col =  label
processing ext_file: Ext_rm_sim_0_8.csv; feat: InfoContent
e_roc_auc =  0.5050234741784038
e_roc_auc =  0.5122535211267606
e_roc_auc =  0.4963849765258216
e_roc_auc =  0.462112676056338
e_roc_auc =  0.4815492957746479
values_col =  label
processing ext_file: Ext_rm_sim_0_9.csv; feat: MorganFP
e_roc_auc =  0.7091797272542235
e_roc_auc =  0.8074089151231427
e_roc_auc =  0.838143700386729
e_roc_auc =  0.7134337472013027
e_roc_auc =  0.8067575819255038
values_col =  label
processing ext_file: Ext_rm_sim_0_9.csv; feat: RDkitFP
e_roc_auc =  0.5566252798697334
e_roc_auc =  0.5722979849379198
e_roc_auc =  0.5274374109505394
e_roc_auc =  0.5989212293914106
e_roc_auc =  0.572501526562182
values_col =  label
processing ext_file: Ext_rm_sim_0_9.csv; feat: AtomPairFP
e_roc_auc =  0.8392021168328925
e_roc_auc =  0.902829228

In [11]:
num_folds = 5;
ext_file = 'Ext.csv'
ext_base = ext_file.replace('.csv','')
for feature in fingerprint_list+descriptor_list:
    X_e,y_e = get_ext_Xye_fp_desc(ext_file,feature)
    print(f'processing ext_file: {ext_file}; feat: {feature}')
    load_svm_model_and_predict(feature,num_folds)

values_col =  label
processing ext_file: Ext.csv; feat: MorganFP
e_roc_auc =  0.7301129888313633
e_roc_auc =  0.8134707433818502
e_roc_auc =  0.8417667936569958
e_roc_auc =  0.7355670606623033
e_roc_auc =  0.8167269056689784
values_col =  label
processing ext_file: Ext.csv; feat: RDkitFP
e_roc_auc =  0.5739962879749927
e_roc_auc =  0.5877698544495457
e_roc_auc =  0.545602552831233
e_roc_auc =  0.6196802448634039
e_roc_auc =  0.5977337110481586
values_col =  label
processing ext_file: Ext.csv; feat: AtomPairFP
e_roc_auc =  0.853716909250757
e_roc_auc =  0.9062062453192666
e_roc_auc =  0.8614340138712514
e_roc_auc =  0.8965680049493667
e_roc_auc =  0.8554752368858063
values_col =  label
processing ext_file: Ext.csv; feat: TorsionFP
e_roc_auc =  0.6192081013317703
e_roc_auc =  0.5924750089544464
e_roc_auc =  0.6819380677932989
e_roc_auc =  0.5380970987594021
e_roc_auc =  0.5549965810295985
values_col =  label
processing ext_file: Ext.csv; feat: AvalonFP
e_roc_auc =  0.7083618247533456
e_r

# rf

In [14]:
def load_rf_model_and_predict(feature,num_folds):  
    for i in range(num_folds): 
        model_file=f"rf/model_{feature}_fold{i}.job"
        rf_clf = joblib.load(model_file)
        y_prob_e=rf_clf.predict_proba(X_e)[:,1]  ## no need other parameters here
        e_roc_auc = ROC_AUC(y_e,y_prob_e)
        print('e_roc_auc = ', e_roc_auc)
        if not os.path.exists('rf'):os.mkdir('rf')
        pd.DataFrame(y_prob_e,columns=['prob']).to_csv(f'rf/p_{ext_base}_{feature}_fold{i}.csv',index=False)
    return

In [15]:
num_folds = 5;
ext_files = ['Ext_rm_sim_0_6.csv','Ext_rm_sim_0_7.csv','Ext_rm_sim_0_8.csv','Ext_rm_sim_0_9.csv']
for ext_file in ext_files:
    ext_base = ext_file.replace('.csv','')
    for feature in fingerprint_list+descriptor_list:
        X_e,y_e = get_ext_rm_sim_Xye_fp_desc(ext_file,feature)
        print(f'processing ext_file: {ext_file}; feat: {feature}')
        load_rf_model_and_predict(feature,num_folds)

values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: MorganFP
e_roc_auc =  0.11111111111111116
e_roc_auc =  0.11111111111111116
e_roc_auc =  0.11111111111111116
e_roc_auc =  0.0740740740740741
e_roc_auc =  0.0740740740740741
values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: RDkitFP
e_roc_auc =  0.11111111111111116
e_roc_auc =  0.11111111111111116
e_roc_auc =  0.11111111111111116
e_roc_auc =  0.11111111111111116
e_roc_auc =  0.11111111111111116
values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: AtomPairFP
e_roc_auc =  0.11111111111111116
e_roc_auc =  0.11111111111111116
e_roc_auc =  0.11111111111111116
e_roc_auc =  0.11111111111111116
e_roc_auc =  0.11111111111111116
values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: TorsionFP
e_roc_auc =  0.0
e_roc_auc =  0.0
e_roc_auc =  0.0
e_roc_auc =  0.03703703703703705
e_roc_auc =  0.0
values_col =  label
processing ext_file: Ext_rm_sim_0_6.csv; feat: AvalonFP
e_roc_auc =  0.074074074

e_roc_auc =  0.31631578947368416
values_col =  label
processing ext_file: Ext_rm_sim_0_7.csv; feat: MHFP6
e_roc_auc =  0.5252631578947369
e_roc_auc =  0.49526315789473685
e_roc_auc =  0.4936842105263158
e_roc_auc =  0.5536842105263158
e_roc_auc =  0.4942105263157895
values_col =  label
processing ext_file: Ext_rm_sim_0_7.csv; feat: MAP4
e_roc_auc =  0.7073684210526314
e_roc_auc =  0.6484210526315789
e_roc_auc =  0.6863157894736842
e_roc_auc =  0.6178947368421052
e_roc_auc =  0.6957894736842105
values_col =  label
processing ext_file: Ext_rm_sim_0_7.csv; feat: Property
e_roc_auc =  0.41052631578947374
e_roc_auc =  0.3473684210526316
e_roc_auc =  0.42000000000000004
e_roc_auc =  0.3831578947368421
e_roc_auc =  0.36736842105263157
values_col =  label
processing ext_file: Ext_rm_sim_0_7.csv; feat: Constitution
e_roc_auc =  0.25842105263157894
e_roc_auc =  0.32473684210526316
e_roc_auc =  0.2826315789473684
e_roc_auc =  0.2636842105263158
e_roc_auc =  0.24473684210526317
values_col =  label

e_roc_auc =  0.7091079812206573
values_col =  label
processing ext_file: Ext_rm_sim_0_8.csv; feat: Topology
e_roc_auc =  0.7212676056338028
e_roc_auc =  0.6957746478873238
e_roc_auc =  0.6204225352112676
e_roc_auc =  0.6138967136150235
e_roc_auc =  0.6100469483568075
values_col =  label
processing ext_file: Ext_rm_sim_0_8.csv; feat: Kappa
e_roc_auc =  0.49854460093896713
e_roc_auc =  0.4789671361502348
e_roc_auc =  0.43258215962441315
e_roc_auc =  0.4682159624413146
e_roc_auc =  0.4707511737089202
values_col =  label
processing ext_file: Ext_rm_sim_0_8.csv; feat: Path
e_roc_auc =  0.6125821596244132
e_roc_auc =  0.6104694835680752
e_roc_auc =  0.5894366197183099
e_roc_auc =  0.5902347417840377
e_roc_auc =  0.6048826291079812
values_col =  label
processing ext_file: Ext_rm_sim_0_8.csv; feat: Matrix
e_roc_auc =  0.7352582159624412
e_roc_auc =  0.7004225352112675
e_roc_auc =  0.590469483568075
e_roc_auc =  0.6556338028169014
e_roc_auc =  0.7255399061032863
values_col =  label
processing e